In [7]:
%pip install optuna cebra==0.4.0 matplotlib==3.9.2 numpy pandas==2.2.3 scipy==1.14.1 seaborn==0.13.2 umap_learn pyspark==3.5.2 python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 16.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cebra
import torch
import torch.utils
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from utils import TensorDataset, SimpleTensorDataset, SupervisedNNSolver, RDDDataset
import dotenv
import os
dotenv.load_dotenv()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'utils'

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Initialize a SparkSession
spark = SparkSession.builder \
    .master("spark://131.220.127.46:7077") \
    .appName("UranusCluster") \
    .config("spark.executor.memory", "10g") \
    .getOrCreate()

# Verify the SparkContext
print(spark.sparkContext.getConf().getAll())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 18:09:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data Load

In [ ]:
# Load parquet files from the folder given by the environment variable $DATA_PATH
parquet_dir = os.environ.get("DATA_PATH")
df = spark.read.format("parquet").load(parquet_dir)\
    .select(["index","neural_data", "positional_encoding", "file_name"])
# df_iterator = df.rdd.mapPartitions(lambda part: [list(part)]).toLocalIterator()
df.show(5)

24/12/14 14:51:20 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: sftp://131.220.127.56/data/transformed.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "sftp"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.sc

Py4JJavaError: An error occurred while calling o149.load.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "sftp"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(

In [ ]:
n_neurons = max([row.neural_size for row in df.select(F.max(F.size("neural_data")).alias("neural_size")).collect()])
n_rows = df.count()

24/12/14 13:58:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
kf = KFold(n_splits = 3)
for train_indices, test_indices in kf.split(np.arange(n_rows)):
    train_indices = spark.createDataFrame(np.array(train_indices).reshape(-1,1).tolist(), "index: int")
    test_indices =  spark.createDataFrame(np.array(test_indices).reshape(-1,1).tolist(), "index: int")
    df_train = df.join(train_indices, ['index'], how='right')
    df_test  = df.join(test_indices, ['index'], how='right')
    break
# df_train.show()

In [ ]:
df_train.show(5)

24/12/14 14:00:53 WARN Utils: Suppressing exception in finally: No space left on device
java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:367)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:125)
	at java.base/java.io.BufferedOutputStream.implWrite(BufferedOutputStream.java:222)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:206)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.finish(LZ4BlockOutputStream.java:257)
	at net.jpountz.lz4.LZ4BlockOutputStream.close(LZ4BlockOutputStream.java:195)
	at org.apache.spark.storage.DiskBlockObjectW

KeyboardInterrupt: 

In [ ]:
delta = 5000
def get_x_ticks(L:int):
    x_ticks = np.arange(0,L,delta)
    x_tick_labels = [f"{t[i]/100:.2f}" for i in x_ticks]
    return x_ticks, x_tick_labels

In [ ]:
"""
- Choose number of folds
- Shuffle data?
- Get indices for test and train
- Iterate over partitions
"""

# CEBRA Encoder

# Models

In [ ]:
class ModelPipeline:
    def __init__(self,
        model_name,
        train_iterator,
        test_iterator,
        num_units,
        latent_dimension = 8):
        self.model_name = model_name
        self.train_iterator = train_iterator
        self.test_iterator = test_iterator
        self.latent_dimension = latent_dimension
        self.num_units = num_units
        with device:
            self.model = cebra.models.init(
                name = model_name,
                num_neurons = self.train_dataset.neural.shape[1],
                num_units = num_units,
                num_output = latent_dimension
            )
            self.train_dataset.configure_for(self.model)
            self.test_dataset.configure_for(self.model)
    def train_embedding(self, learning_rate, batch_size, steps = 1000, verbose = True):
        self.batch_size = batch_size
        self.steps = steps
        self.learning_rate = learning_rate
        self.steps = steps
        with device:
            criterion = cebra.models.criterions.LearnableCosineInfoNCE()
            optimizer = torch.optim.Adam(
                list(self.model.parameters()) + list(criterion.parameters()),
                lr = learning_rate
            )
            self.embedding_solver = cebra.solver.SingleSessionSolver(
                model = self.model,
                criterion = criterion,
                optimizer = optimizer,
                tqdm_on = verbose
            )
            train_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.train_dataset,
                num_steps = steps,
                batch_size = batch_size,
                prior = "empirical"
            )
            self.embedding_solver.fit(loader=train_loader)
    def train_decoder(self, verbose = True):
        with device:
            self.simple_train_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_train_tensor.type(torch.FloatTensor),
                    labels = self.y_train_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                ),
                batch_size = self.batch_size,
                shuffle = True
            )
            self.binaryClassifier = torch.nn.Sequential(
                self.model,
                torch.nn.Linear(self.latent_dimension,self.latent_dimension),
                torch.nn.GELU(),
                torch.nn.Linear(self.latent_dimension,1),
                torch.nn.GELU(),
                torch.nn.Linear(1,1) #Logit output
            ).to(device)
            self.decoder_solver = SupervisedNNSolver(
                model = self.binaryClassifier,
                criterion = torch.nn.BCEWithLogitsLoss(),
                optimizer = torch.optim.Adam(self.binaryClassifier.parameters(), lr = self.learning_rate)
            )
            self.decoder_solver.fit(self.simple_train_dataloader, num_steps = self.steps)
    def train(self, learning_rate, batch_size, steps, verbose = True):
        self.train_embedding(learning_rate, batch_size, steps, verbose)
        self.train_decoder(verbose = verbose)
    def score(self, verbose = True):
        with device:
            test_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.test_dataset,
                num_steps = self.steps,
                batch_size = self.batch_size,
                prior = "empirical"
            )
            self.embedding_score = self.embedding_solver.validation(test_loader)
            self.simple_test_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_test_tensor.type(torch.FloatTensor),
                    labels = self.y_test_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                    ),
                batch_size = self.batch_size,
                shuffle = True
                )
            self.decoder_score = self.decoder_solver.validation(self.simple_test_dataloader)['total']
            try:
                # Fit an exponential decay to the history and get the R2 value
                def exp_decay(x, a, b, c):
                    return a * np.exp(-b * x) + c

                # Fit the exponential decay to the solver history
                x_data = np.arange(len(self.embedding_solver.history))
                y_data = self.embedding_solver.history
                y0 = y_data[0]
                yf = y_data[-1]
                popt, _ = curve_fit(exp_decay, x_data, y_data, p0=(y0 - yf, 1e-6, yf))

                # Calculate the R2 value
                y_pred = exp_decay(x_data, *popt)
                r2 = r2_score(y_data, y_pred)
                if verbose:
                    print(f"Expontial decay R2: {r2}, embedding score: {self.embedding_score}, decoder score: {self.decoder_score}")
                return -r2 + self.embedding_score + 2*self.decoder_score
            except:
                return float("inf")
        

In [ ]:
pipeline = ModelPipeline(
    "offset36-model",
    X_train_tensor=X_train_tensor,
    y_train_tensor=y_train_tensor,
    X_test_tensor=X_test_tensor,
    y_test_tensor=y_test_tensor,
    latent_dimension=8,
    num_units=2
)
pipeline.train(learning_rate = 1e-3, batch_size = 12, steps = 10)
pipeline.score()

pos: -0.9850 neg:  3.4666 total:  2.4816 temperature:  0.9968: 100%|██████████| 10/10 [00:01<00:00,  5.97it/s]
100%|██████████| 10/10 [00:00<00:00, 21.33it/s]


Expontial decay R2: 0.1465373380689179, embedding score: 2.484728240966797, decoder score: 0.8064513206481934


3.951093544194266

In [ ]:
models_to_try = ('offset10-model', 'offset36-model', 'offset1-model')
models_to_try

('offset10-model', 'offset36-model', 'offset1-model', 'offset5-model')

In [ ]:
def experiment(trial):
    model_name = trial.suggest_categorical("model_name", models_to_try)
    num_units = trial.suggest_int("num_units", 1, X_train.shape[1])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int("batch_size", 50, 256, log=True)
    model = ModelPipeline(
        model_name,
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test_tensor=y_test_tensor,
        num_units=num_units,
        latent_dimension=8
    )
    model.train(learning_rate, batch_size, 500)
    score = model.score()
    
    # Save the best model to disk
    if not hasattr(experiment, "best_score") or score < experiment.best_score:
        experiment.best_score = score
        experiment.pipeline = model
        torch.save(model.model.state_dict(), "./data/models/best_model.pth")
    
    return score

In [ ]:
study = optuna.create_study(storage="sqlite:///data/optuna.db", study_name="cebra_offsets", direction="minimize", load_if_exists=True)
study.optimize(experiment, n_trials=100)

study.best_params

[I 2024-11-29 15:16:13,326] Using an existing study with name 'cebra_offsets' instead of creating a new one.
pos: -0.9735 neg:  5.9531 total:  4.9797 temperature:  0.9370: 100%|██████████| 500/500 [04:10<00:00,  2.00it/s]
[W 2024-11-29 15:20:34,993] Trial 1 failed with parameters: {'model_name': 'offset5-model', 'num_units': 229, 'learning_rate': 0.00016581411988856763, 'batch_size': 163} because of the following error: RuntimeError('Input type (double) and bias type (float) should be the same').
Traceback (most recent call last):
  File "/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_91719/3262429375.py", line 15, in experiment
    model.train(learning_rate, batch_size, 500)
  File "/tmp/ipykernel_91719/2048962979.py", line 79, in train
    self.train_decoder(verbose = verbose)
  File "/tmp/ipykernel

RuntimeError: Input type (double) and bias type (float) should be the same